In [1]:
cd ..

/home/ubuntu/anthony/anthony-env/Project/KG-Link_Prediction


In [2]:
#Code based on https://www.dgl.ai/news/2020/06/09/covid.html

In [3]:
from lib import format_transformer

In [4]:
import pandas as pd
import numpy as np

drkg_file = './data/v1_res/relation/DDires.csv'
df = pd.read_csv(drkg_file)
df.head()

,Drug,Disease,Palliates_Hetionet,Treats_Hetionet,Effect_KEGG,Association_CTD
0,DrugBank:DB00997,DOID:363,0,1,0,1
1,DrugBank:DB00206,DOID:10763,0,1,0,1
2,DrugBank:DB00960,DOID:10763,0,1,0,1
3,DrugBank:DB00665,DOID:10283,0,1,1,1
4,DrugBank:DB00290,DOID:2998,0,1,0,1


In [11]:
triples = format_transformer.transform_kg_csv_to_triplets([drkg_file], relation_sample_ratio=.05)

In [12]:
triples[0:5]

[['DrugBank:DB09389', 'Association_CTD', 'DOID:4989'],
 ['MESH:C049325', 'Association_CTD', 'DOID:10283'],
 ['MESH:D008555', 'Association_CTD', 'DOID:0060647'],
 ['MESH:C026942', 'Association_CTD', 'DOID:1287'],
 ['MESH:D015306', 'Association_CTD', 'DOID:4964']]

In [13]:
len(triples)

134263

In [14]:
format_transformer.partition_dataset(triples, "WANGKG", .9, .05)

In [17]:
!DGLBACKEND=pytorch dglke_train --dataset WANGKG --data_path ./notebooks/train \
                               --data_files WANGKG_train.tsv WANGKG_valid.tsv WANGKG_test.tsv \
                               --format 'raw_udd_hrt' --model_name TransE_l2 --batch_size 2048 \
                               --neg_sample_size 256 --hidden_dim 400 --gamma 12.0 --lr 0.1 \
                               --max_step 100000 --log_interval 1000 --batch_size_eval 16 -adv \
                               --regularization_coef 1.00E-07 --test --num_thread 1 \
                               --gpu 0 1 2 3 4 5 6 7 --num_proc 8 \
                               --neg_sample_size_eval 10000 --async_update

Using backend: pytorch
Reading train triples....
Finished. Read 120836 train triples.
Reading valid triples....
Finished. Read 6713 valid triples.
Reading test triples....
Finished. Read 6714 test triples.
|Train|: 120836
random partition 120836 edges into 8 parts
part 0 has 15105 edges
part 1 has 15105 edges
part 2 has 15105 edges
part 3 has 15105 edges
part 4 has 15105 edges
part 5 has 15105 edges
part 6 has 15105 edges
part 7 has 15101 edges
/home/ubuntu/anthony/anthony-env/lib/python3.6/site-packages/dgl/base.py:45: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  return warnings.warn(message, category=category, stacklevel=1)
/home/ubuntu/anthony/anthony-env/lib/python3.6/site-packages/dgl/base.py:45: DGLWarning: Keyword arguments ['readonly', 'multigraph', 'sort_csr'] are deprecated in v0.5, and can be safely removed in all cases.
  return warnings.warn(message, category=category, stacklevel=1)
Traceback (most recent call last):
  File 